In [1]:
# settings of api request

import requests
baseUrl = 'http://www.tng-project.org/api/'
headers = {"api-key":"6a883b0dea91aa886c3dbc7b9456d9f3"} # use my api-key or enter yours

def get(path, params=None):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically

    if 'content-disposition' in r.headers:
        filename = r.headers['content-disposition'].split("filename=")[1]
        with open(filename, 'wb') as f:
            f.write(r.content)
        return filename # return the filename string

    return r

In [2]:
# import 
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

In [3]:
# set the cosmology you prefer
from astropy.cosmology import Planck15
cosmo = Planck15
h =  cosmo.H(5).value/100

In [4]:
r = get(baseUrl)

In [5]:
# get total snapshot numbers
names = [sim['name'] for sim in r['simulations']]
simname = 'TNG100-1'
i = names.index(simname)
sim = get( r['simulations'][i]['url'] )
snaps = get( sim['snapshots'] )
len(snaps)
#print(snaps)

100

In [10]:
# get redshift of "redshift 0"
snap_number = 17
z0 = snaps[snap_number]['redshift']
from astropy.coordinates import Distance
import astropy.units as u
z0

4.99593346816462

In [11]:
# convert ckpc/h to kpc
def comv_to_phys(R_comv, z):
    R_phys=R_comv/(1+z)
    R_phys = R_phys/h
    return R_phys

In [23]:
subid=278 #enter the subhalo id

In [24]:
import h5py
start_url = "http://www.tng-project.org/api/%s/snapshots/"%simname + str(snap_number) + "/subhalos/" + str(subid)
print(start_url)
sub_prog_url = start_url
sub_prog = get(sub_prog_url)
sub_prog['pos_x'], sub_prog['pos_y'], sub_prog['pos_z']

http://www.tng-project.org/api/TNG100-1/snapshots/17/subhalos/278


(10914.7, 29171.3, 72776.0)

In [25]:
cutout_request = {'gas':'Coordinates,SubfindHsml,Masses,GFM_Metallicity,StarFormationRate'} # do not add space after comma
cutout = get(sub_prog_url+"/cutout.hdf5", cutout_request)

In [26]:
sub_prog_url+"/cutout.hdf5"

'http://www.tng-project.org/api/TNG100-1/snapshots/17/subhalos/278/cutout.hdf5'

In [27]:
with h5py.File(cutout,'r') as f:
    x = comv_to_phys(f['PartType0']['Coordinates'][:,0] - sub_prog['pos_x'], z0)
    y = comv_to_phys(f['PartType0']['Coordinates'][:,1] - sub_prog['pos_y'], z0)
    z = comv_to_phys(f['PartType0']['Coordinates'][:,2] - sub_prog['pos_z'], z0)
    hsml = comv_to_phys(f['PartType0']['SubfindHsml'][:], z0)
    mass = f['PartType0']['Masses'][:]*10**10
    Z = f['PartType0']['GFM_Metallicity'][:]
    SFR=f['PartType0']['StarFormationRate'][:]


In [28]:
data = np.vstack((x,y,z,hsml, mass, Z))

In [29]:
hdr = "Column 1: x-coordinate (kpc)\n\
Column 2: y-coordinate (kpc)\n\
Column 3: z-coordinate (kpc)\n\
Column 4: smoothing length (kpc)\n\
Column 5: gas mass (Msun)\n\
Column 6: metallicity (1)"
np.savetxt("./%s_gas_id%d.txt"%(simname, subid), data.T, fmt='%.6e', header = hdr)   #change the path

In [30]:
# Cut out stellar particle
cutout_request = {'stars':'Coordinates,SubfindHsml,StellarHsml,GFM_InitialMass,GFM_Metallicity,GFM_StellarFormationTime'}
cutout = get(sub_prog_url+"/cutout.hdf5", cutout_request)

In [31]:
with h5py.File(cutout,'r') as f:
    x = comv_to_phys(f['PartType4']['Coordinates'][:,0] - sub_prog['pos_x'], z0)
    y = comv_to_phys(f['PartType4']['Coordinates'][:,1] - sub_prog['pos_y'], z0)
    z = comv_to_phys(f['PartType4']['Coordinates'][:,2] - sub_prog['pos_z'], z0)
    hsml = comv_to_phys(f['PartType4']['StellarHsml'][:], z0)
    imass = f['PartType4']['GFM_InitialMass'][:]*10**10
    Z = f['PartType4']['GFM_Metallicity'][:]
    age = Planck15.lookback_time(1/f['PartType4']['GFM_StellarFormationTime'][:]-1)-Planck15.lookback_time(z0)
    age = age.value
    print(age)


[0.12746969 0.01795846 0.00441808 ...        nan        nan        nan]


In [32]:
data = np.vstack((x,y,z,hsml, imass, Z, age))

hdr = "Column 1: x-coordinate (kpc)\n\
Column 2: y-coordinate (kpc)\n\
Column 3: z-coordinate (kpc)\n\
Column 4: smoothing length (kpc)\n\
Column 5: initial mass (Msun)\n\
Column 6: metallicity (1)\n\
Column 7: age (Gyr)"
np.savetxt("./%s_star_id%d.txt"
%(simname, subid), data.T, fmt='%.6e', header = hdr) #change the path
stars = np.loadtxt("./%s_star_id%d.txt"%(simname, subid))
RMList = []
for i in range(np.shape(stars)[0]):
    if np.isnan(stars[i,6]) :
        RMList.append(i)
stars = np.delete(stars,RMList,0)
np.savetxt("./%s_star_id%d.txt"
%(simname, subid), stars, fmt='%.6e', header = hdr)

In [33]:
stars = np.loadtxt('TNG100-1_star_id845.txt')
#print(np.shape(stars)[0])
RMList = []

for i in range(np.shape(stars)[0]):
    if np.isnan(stars[i,6]) :
        RMList.append(i)
    else:
        a=1

#print(RMList)
stars = np.delete(stars,RMList,0)
#print(np.shape(stars)[0])
np.savetxt('TNG100-1_star_id845.txt', stars)